In [51]:
!accelerate launch run_sign2vec_pretraining.py \
						--model_name_or_path="patrickvonplaten/wav2vec2-base-v2" \
						--output_dir="./sign2vec" \
						--max_train_steps="2" \
						--num_warmup_steps="3" \
						--gradient_accumulation_steps="4" \
						--learning_rate="0.001" \
						--weight_decay="0.01" \
						--max_duration_in_seconds="20.0" \
						--min_duration_in_seconds="2.0" \
						--logging_steps="1" \
						--saving_steps="10000" \
						--per_device_train_batch_size="8" \
						--per_device_eval_batch_size="8" \
						--adam_beta1="0.9" \
						--adam_beta2="0.98" \
						--adam_epsilon="1e-06" \
						--gradient_checkpointing \
						--mask_time_prob="0.65" \
						--mask_time_length="10" \
						--use_face \
						--use_hands \
						--use_pose \
						--train_info_path="../sign2vec/config/info.json" \
						--train_data_path="../sign2vec/features" \
						--validation_info_path="../sign2vec/config/info.json" \
						--validation_data_path="../sign2vec/features" \
						--config_name="config.json"

W0615 14:06:08.267000 8404925120 torch/distributed/elastic/multiprocessing/redirects.py:27] NOTE: Redirects are currently not supported in Windows or MacOs.
Traceback (most recent call last):
  File "/Users/karahansahin/Documents/Research/sign2vec/pretraining/run_sign2vec_pretraining.py", line 64, in <module>
Traceback (most recent call last):
  File "/Users/karahansahin/Documents/Research/sign2vec/pretraining/run_sign2vec_pretraining.py", line 64, in <module>
        main()
main()
  File "/Users/karahansahin/Documents/Research/sign2vec/pretraining/run_sign2vec_pretraining.py", line 30, in main
  File "/Users/karahansahin/Documents/Research/sign2vec/pretraining/run_sign2vec_pretraining.py", line 30, in main
    accelerator, api, repo_id = initialize_accelerator(args)
              accelerator, api, repo_id = initialize_accelerator(args) 
                                             ^^^^^^^^    ^ ^ ^ ^ ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^  File "/Users/karahansahin/Documents/Research/s